In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp -rf /content/drive/MyDrive/colab/didimdol/* .
!unzip  /content/dataset/dataset.zip -d ./dataset

In [1]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.applications import resnet50
from tensorflow.keras import preprocessing
from tensorflow.keras.callbacks import ModelCheckpoint

# 모델 로딩 및 binary classification 세팅

In [2]:
model = resnet50.ResNet50(weights=None, classes = 1, classifier_activation='sigmoid')

# 학습데이터 셋업

In [3]:
trn_data_dir = 'dataset/casting_data/train'
val_data_dir = 'dataset/casting_data/test'
batch_size = 64
img_height = 224
img_width = 224

In [4]:
train_ds = preprocessing.image_dataset_from_directory(
    trn_data_dir,
    label_mode = 'binary',
    seed=123,
    shuffle=True,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    val_data_dir,
    #validation_split=0.2,
    label_mode = 'binary',
    #subset="validation",
    shuffle=False,
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

Found 6633 files belonging to 2 classes.
Found 715 files belonging to 2 classes.


In [7]:
data_augmentation = keras.Sequential(
  [
    keras.layers.experimental.preprocessing.Rescaling(1./255),
    keras.layers.experimental.preprocessing.Normalization(mean=[0.485, 0.456, 0.406], variance=[0.229, 0.224, 0.225]),
    keras.layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=(img_height, img_width, 3)),
    keras.layers.experimental.preprocessing.RandomFlip("vertical",   input_shape=(img_height, img_width, 3)),
  ]
)

test_processing = keras.Sequential(
  [
    keras.layers.experimental.preprocessing.Rescaling(1./255),
    keras.layers.experimental.preprocessing.Normalization(mean=[0.485, 0.456, 0.406], variance=[0.229, 0.224, 0.225])
  ]
)

normalized_trn_ds = train_ds.map(lambda x, y: (data_augmentation(x), y))
normalized_test_ds = test_ds.map(lambda x, y: (test_processing(x), y))

# 모델 학습 셋업

In [8]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.BinaryAccuracy()])

In [9]:
checkpoint = ModelCheckpoint('check_points/resnet50/model.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

# 모델 학습

In [10]:
epochs=20
history = model.fit(
  normalized_trn_ds,
  validation_data=normalized_test_ds,
  epochs=epochs,
  callbacks=callbacks_list
)

Epoch 1/20
104/104 [==============================] - 59s 505ms/step - loss: 0.3618 - binary_accuracy: 0.8961 - val_loss: 1.2958 - val_binary_accuracy: 0.4210

Epoch 00001: val_loss improved from inf to 1.29584, saving model to check_points/resnet50/model.h5


/usr/local/lib/python3.6/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/20
104/104 [==============================] - 51s 487ms/step - loss: 0.1081 - binary_accuracy: 0.9616 - val_loss: 2.5459 - val_binary_accuracy: 0.6294

Epoch 00002: val_loss did not improve from 1.29584
Epoch 3/20
104/104 [==============================] - 51s 486ms/step - loss: 0.0610 - binary_accuracy: 0.9809 - val_loss: 12.3210 - val_binary_accuracy: 0.6336

Epoch 00003: val_loss did not improve from 1.29584
Epoch 4/20
104/104 [==============================] - 51s 489ms/step - loss: 0.0366 - binary_accuracy: 0.9894 - val_loss: 0.5074 - val_binary_accuracy: 0.8685

Epoch 00004: val_loss improved from 1.29584 to 0.50738, saving model to check_points/resnet50/model.h5
Epoch 5/20
104/104 [==============================] - 51s 489ms/step - loss: 0.0388 - binary_accuracy: 0.9891 - val_loss: 0.6076 - val_binary_accuracy: 0.8056

Epoch 00005: val_loss did not improve from 0.50738
Epoch 6/20
104/104 [==============================] - 51s 487ms/step - loss: 0.0284 - binary_accuracy: 